<a href="https://colab.research.google.com/github/esuda/mineiracao_dados_complexos/blob/master/Big%20Data/Big_Data_010_Trabalho_2_Sentimento_Noriaki_Suda_Wakabayashi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pacote de dados


*   Abra o link do pacote de dados: https://tinyurl.com/bd10-tweets
*   Selecione a opção "Adicionar Atalho ao Drive"

In [ ]:
!pip install mrjob nltk twython

     |████████████████████████████████| 439 kB 5.5 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%file trab2.py

from mrjob.job import MRJob
from mrjob.step import MRStep
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import re

class Sentiment(MRJob):

  def mapper_init(self):
    nltk.download('vader_lexicon', quiet=True)

  def mapper(self, _, line):
    yield None, [0, 1]

  def combiner(self, key, values):
    yield None, [0, 1]

  def reducer(self, key, values):
    yield None, 0

if __name__ == '__main__':
    Sentiment.run()

Writing trab2.py


In [ ]:
!python trab2.py -r local "/content/drive/My Drive/twitter_cs/sample_clean.csv" 

No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /tmp/trab2.root.20211030.111352.846274
Running step 1 of 1...
job output is in /tmp/trab2.root.20211030.111352.846274/output
Streaming final output from /tmp/trab2.root.20211030.111352.846274/output...
null	0
Removing temp directory /tmp/trab2.root.20211030.111352.846274...


In [ ]:
!python trab2.py -r local "/content/drive/My Drive/twitter_cs/sample_clean_larger.csv.gz" 

In [ ]:
!python trab2.py -r local "/content/drive/My Drive/twitter_cs/twcs_clean.csv.gz" 